In [4]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [5]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [6]:
attcs = ATTCS()

In [7]:
await attcs.start_task

Read historical data in 1.94 sec
Read historical data in 2.20 sec
Read historical data in 2.73 sec
Read historical data in 3.35 sec
Read historical data in 3.99 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 55 messages
Read historical data in 4.87 sec
Read historical data in 6.03 sec
RemoteTelemetry(ATDome, 0, position) falling behind; read 26 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 16 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 36 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 17 messages


In [28]:
await attcs.enable(settings={
    'ataos': "tolerance",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

Gathering settings.
Received settings from users.: {'ataos': 'tolerance', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Settings versions: {'ataos': 'tolerance', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'auxtel_201912_crazy.yaml', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Enabling all components
Enabling  atmcs
Enabling  atptg
Enabling  ataos
Enabling  atpneumatics
Enabling  athexapod
Enabling  atdome
Enabling  atdometrajectory
[atmcs]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atptg]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ataos]::[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.ENABLED: 2>]
[atdome]::[<State.ENABLED: 2>]
[atdometrajectory]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components enabled.


In [ ]:
await attcs.startup(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
await attcs.atmcs.cmd_setInstrumentPort.set_start(port=0)

In [ ]:
await attcs.atpneumatics.cmd_openM1CellVents.start()

In [34]:
await asyncio.sleep(5)
await asyncio.gather(salobj.set_summary_state(attcs.atmcs, salobj.State.STANDBY, timeout=120),
                     salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.ataos, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atpneumatics, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.athexapod, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atdometrajectory, salobj.State.STANDBY, timeout=30),
                    return_exceptions=True)

[[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.FAULT: 3>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>],
 [<State.STANDBY: 5>]]

In [ ]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED, timeout=30)

In [ ]:
await attcs.slew_icrs(ra="03 58 01.7669489", dec="-13 30 30.669770", target_name="gam-Eri")

In [ ]:
HD 20894

In [ ]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="03 21 24.0043600176", dec="-23 38 06.552850356", rot_pa=120., target_name="HD20894")

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.atptg.cmd_stopTracking.start()

In [12]:
await salobj.set_summary_state(attcs.ataos, salobj.State.OFFLINE, timeout=30)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.OFFLINE: 4>]

RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 10 messages


In [13]:
await salobj.set_summary_state(attcs.ataos, salobj.State.ENABLED, timeout=30, settingsToApply="tolerance")

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [23]:
await attcs.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True)

RemoteEvent(ATMCS, 0, target) falling behind; read 55 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 24 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 24 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 24 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 25 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 26 messages
RemoteEvent(AT

In [16]:
await attcs.ataos.cmd_setLogLevel.set_start(level=10)

RemoteEvent(ATAOS, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATAOS, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 38 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 23 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 22 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 23 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 26 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) fal

In [29]:
await salobj.set_summary_state(attcs.atdometrajectory, salobj.State.STANDBY, timeout=30)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [30]:
await attcs.atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=75.)

In [32]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=attcs.dome_park_az)

In [33]:
await attcs.atptg.cmd_stopTracking.start()

In [26]:
await attcs.shutdown()

Disabling ATAOS corrections
Close telescope cover.
Disable ATDomeTrajectory
Close dome.


AckTimeoutError: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=146374207, ack=<SalRetCode.CMD_INPROGRESS: 301>, error=0, result='')

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 99 messages
